# Notebook 1 - Working with tabulated data 
----------------------------------------------------------------

## Table of Content <a id='toc' />

1. **[DataFrame subsetting](#3)**  
    1.1 [Accessing specific rows and columns](#31)  
    1.2 [Conditional selection](#32)  
    <br>
    
2. **[Operations on columns](#4)**  
    2.1 [Arithmetic operations](#41)  
    2.2 [Applying built-in summary functions](#42)  
    2.3 [Applying custom functions](#43)  
    <br>

3. **[Grouping data by factor](#a3)**

4. **[Writing DataFrames to disk](#5)**
   <br>

**Supplementary Material - [Additional topics, not covered in class](#a)**  
  * Annex 1 - [Sorting operations on dataframes](#a1)  
  * Annex 2 - [Dropping rows with missing values](#a2)  
  
  * Annex 4 - [Creating DataFrames and Series from scratch](#a4)  
  * Annex 5 - [Concatenate, merge and join DataFrames](#a5)  
  * Annex 6 - [Mixed selection by names and positions for `.loc[]` and `.iloc[]`](#a6)  
  * Annex 7 - [more ways to add rows to a DataFrame](#a7)
  * Annex 8 - [Wide and long format](#a8)  
  
<br>
<br>

In [ ]:
import pandas as pd

# 1. DataFrame subsetting <a id='3'></a>

## 1.1 Accessing specific rows and columns <a id='31'></a>

A very common operation to perform on *DataFrames* is to **create a subset** by selecting certain rows and/or columns.

There are 2 methods in pandas to perform a selection on a DataFrame (here `df`):
* **`df.loc[<row index values>, <column names>]`** - to select based on row (index) and column **names**.
* **`df.iloc[<row positions>, <column positions>]`** to select based on row and column **positions**.

<br>

### Subsetting a DataFrame with the `.loc[]` indexer

The **`.loc[]` indexer** selects rows and columns based on the **index values of the row** and the **names of the columns**.

In [ ]:
# (Re)Load the titanic dataset as a DataFrame.
df = pd.read_csv("data/titanic.csv")
df.head(3)

In [ ]:
df.loc[2, "Name"]

<br>

**Multiple rows/columns** can be selected by:
* **Passing a sequence** (e.g. `list`, `tuple`) of row/column names: **`.loc[[0, 10], ["Name", "Age"]]`**.
* **Passing a slice** of row/column names: **`.loc[0:10, "Name":"Age"]`**.
  * **`value:`** selects all rows/columns **from `value` till the end**.
  * **`:value`** selects all rows/columns **from the start until `value` (included)**.
  * **`:`** with no values around **selects all rows/columns**.
  
    > *Note:* when selecting on rows only (i.e. select all columns), the `df.loc[<row selection>, ]`
    > and `df.loc[<row selection>]` syntaxes are also possible. The `:` is not compulsory in that case.

<div class="alert alert-block alert-danger">

**Important:** when using *slicing*, the **end element of the slice is included** - unlike every other time in python !

</div>

<br>

**Examples:**

* **Select the first 3 rows** of the columns `Name`, `Age` and `Pclass`.

In [ ]:
df.head()

In [ ]:
df.loc[0:2, ["Name", "Age", "Pclass"]]
df.loc[:2, ["Name", "Age", "Pclass"]]   # Same as above, but omitting the "0" since it's the first index.

<br>

* **Select a range** of columns using *slicing*.

In [ ]:
df.loc[:2, "Name":"Pclass"]  # Reminder: with `.loc[]`, the end element of the slice is included.

<br>

A column can be selected multiple time, in any order. This can be used to **re-order columns**.

In [ ]:
# Selecting all rows, re-arranging columns, and duplicating the "Age" column.
df.loc[:, ["Age", "Pclass", "Name", "Age"]].head()

<br>

**Selecting a single row/column** returns a pandas **Series** object.

In [ ]:
print(type(df.loc[2, ]))
df.loc[2, ]

> **Pandas Series**  
> *Series* are the equivalent of *DataFrame*, but **1-dimensional** (so essentially they
> are a **named vector** of values).  
> 
> Their elements can be accessed in quite a similar way:
>
>    ```py
>    row_5 = df.loc[4,:]
>    print(row_4[0])      # Access an item of a Series by position.
>    print(row_4.Age)     # Access an item of a Series by name.
>    ```

<br>

### Pitfalls with `.loc[]`

**Because the index values often correspond to row positions** (as in our example DataFrame), it is easy to get the wrong impression that `.loc[]` also selects on row positions... but this is not the case.

To illustrate this, let's load a DataFrame with **non-numeric Index values**:

In [ ]:
# Load the titanic data set with the "Name" column as index.
tmp = pd.read_csv("data/titanic.csv", index_col="Name")
tmp.head()

<br>

Let's now try to **select rows 2-5** of our *DataFrame*:

In [ ]:
tmp.loc[1:4, :]  # Oops... this raises a TypeError.

<br>

With `.loc[]`, **we must pass Index/column names**. Positions are not allowed.

In [ ]:
# Select all rows from "Coleff Mr. Peju" to "Dooley Mr. Patrick" (included):
tmp.loc["Coleff Mr. Peju":"Dooley Mr. Patrick", :]

> 🌈 **Note:** you could get around this problem by querying the index at the desired positions:
>
> ```python
> tmp.loc[tmp.index[1]:tmp.index[4], :]
> ```
> <br>
>
> But at this point it's a lot more convenient to **use the `.iloc[]` indexer**
> (see the supplementary material section below for details):
>
> ```python
> tmp.iloc[1:5, :]
> ```

<br>

<div class="alert alert-block alert-info">

### Supplementary material: the `.iloc[]` indexer

The **`.iloc[]` indexer** is very similar to `.loc[]`, except that it selects based on row/column position rather than on index and column names.
* As usual in Python, **position indexes are zero based**, meaning that the first row/column has position 0.
* Unlike `.loc[]`, with **`.iloc[]`, the end index is excluded**, as usual when doing slicing in Python.

  ```python
  df.iloc[0:3, :]  # Selects the first 3 rows.
  df.loc[0:3, :]   # Selects the first 4 rows.
  ```

<br>

**Examples:**

```python
# Select the first 3 rows (positions 0 to 2).
df.iloc[0:3, :]

# Select the 6th, 7th and 8th rows (positions 5 to 7) and the last 2 columns.
df.iloc[5:8, -2:]
```

    
</div>



<br>

<div class="alert alert-block alert-success">
    
### Micro-Exercise 1

Load the `data/titanic.csv` dataset with: `df = pd.read_csv("data/titanic.csv")`.

Then, using the **`.loc[]`** indexer:
* **Select** all rows with odd Index values from the Titanic data frame, and the columns `Name`, `Age` and `Fare`.
* **Re-order** the columns so that `Age` is first and `Name` is second.
* 🎯 **Hint:** we will soon see how to perform conditional selections, but for now you can use the `range()`
  function to help you with this task.  
  Example: `range(0, 21, 3)`  ->  `[0, 3, 6, 9, 12, 15, 18]`

<br>

🔮 **Additional task, if you have time:**
* Perform the same operation, but using the `.iloc[]` indexer. See the supplementary material above for how to use `.iloc[]`.


</div>


<br>
<br>

[Back to ToC](#toc)

## 1.2 Conditional selection <a id='32' />

A powerful way of selecting rows of a DataFrame is by **passing a sequence of boolean values as row selection** to the `.loc[]` indexer. This allows to **filter via conditional selection**.

<div class="alert alert-block alert-warning">
    
**Important:** Making **conditional selections is only supported by the `.loc[]`** indexer. It does *not* work with `.iloc[]`.

> As a workaround for **`.iloc`**, one can call the index of a mask, or use the
  [query method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html).
  E.g. `df.query("Fare > 300")`.

</div>

<br>

**Example:** In the Titanic dataset, select all passengers older than 50 years.

In [ ]:
df = pd.read_csv("data/titanic.csv")

df.loc[df.Age > 50, :]
#df.iloc[df.Age > 50, :]  # Raises a NotImplementedError.

<br>

**Conditional selection** is frequently done by first creating a boolean **mask**, which is then applied to filter the DataFrame:
* A **mask** is a vector of boolean values (`True`/`False`) that indicate whether
  or not a rows satisfies to the defined condition.

<br>

**Example:** crate a masks that filters for women in the Titanic dataset. Here the mask values are `True` if the corresponding value in the `Sex` column of the DataFrame is equal to `female`.

In [ ]:
mask = df["Sex"] == "female"   # This creates a vector (a pandas Series) of boolean values (True/False).
mask.head()

<br>

Now that the mask is created, we can use it to **filter our `DataFrame`** and keep only those rows corresponding to female passengers.

> 🦉 **Reminder:** `.shape[0]` returns the number of rows of a *DataFrame*.

In [ ]:
# Number of female passengers:
df.loc[mask,].shape[0]

In [ ]:
df.loc[mask, "Name":"Pclass"].head()

<br>

### Combining conditions

Selection conditions can be combined to produce more complex selection criteria.  
* Conditions are combined using the **`&`** (logical AND) and **`|`** (logical OR) operators.
* 🔥 **Important:** each conditions must be surrounded by brackets: `( condition 1 ) & ( condition 2 )`.

<br>

**Examples:**

* Select men passengers with a fare > 200.

In [ ]:
df.loc[(df.Sex == "male") & (df.Fare > 200), ].head()

<br>

* Same as above, but in 2 steps: first we create a mask, then we apply it to the *DataFrame*.

In [ ]:
mask = (df.Sex == "male") & (df.Fare > 200) 
df.loc[mask, ].head()

<br>

* Select all people that are **either < 25 or women**.

In [ ]:
df.loc[(df.Age < 25) | (df.Sex == "female"), :]

<br>

### Conditional value assignment

**Row value assignment operations can be combined with a selection operation**, allowing e.g. to modify the values of certain rows in a DataFrame based on a given condition.

Let's imagine that, for some reason, the fares of class 3 passengers are not valid. We want to set them to `NA` values:

In [ ]:
df_copy = df.copy()  # Create a copy of the DataFrame, to avoid modifying the original one.

# NA is represented using pd.NA
df_copy.loc[df.Pclass==3, 'Fare'] = pd.NA
df_copy.head()

<br>

<div class="alert alert-block alert-success">

### Micro-Exercise 2

* From the Titanic dataset, create a mask to select passengers in first class (`Pclass` is `1`)
  that are less than 18 years old.
* What fraction of these passengers survived?  
  🎯 **Hint:** apply the `.mean()` method to a column to compute its mean value.

<br>

🔮 **Additional Task, if you have time:**
* Create a mask to select women and children (< 18 years). How many were onboard the ship?
* Compute the median ticket price (`Fare` column) and age for men and women.
  Use a `for` loop to avoid code duplication (alternatively, you could look into the `.groupby()` method).

<div>


<br>

<div class="alert alert-block alert-info">
    
### Additional material: copy or not copy?

What happens if we select a subset of a DataFrame and modify it? Does the original data stay the same?

This issue is quite complex, but as you are likely to encounter this warning at some point, we here provide a short primer on it. The following link also provides a more [in-depth explanation of view vs. copy](https://www.dataquest.io/blog/settingwithcopywarning).

<div>

In [ ]:
df = pd.read_table("data/titanic.csv", sep=",")
df.head()

In [ ]:
df.loc[df.Sex == "male", "Age"] = 999
df.head()

<div class="alert alert-block alert-info">
    
In the above example, pandas changes the values of the `Age` column in the original `DataFrame` object.

In general, `pandas` avoids doing copies when it can... but let's try another example:

<div>

In [ ]:
# Assign the subset to a new variable.
df_male = df.loc[df.Sex == "male", :]

In [ ]:
# Setting the age to 888 in the subset dataframe of males:
df_male.Age = 888

<div class="alert alert-block alert-info">

What's this? **We get a warning!**
* As can be seen below, the `Age` value for males has been modified in the `df_male` DataFrame, but not in the
  original `df` DataFrame.

<div>

In [ ]:
df_male.head(3)

In [ ]:
df.head(3)

<div class="alert alert-block alert-info">

In this case, the change is made to `df_male` only and not to the original DataFrame `df`.

Sadly, it is not easy to know when you get a **view** or a **copy**.

<br>

![image.png](img/view_copy.png)

<br>

 * **View:** pointer to the original DataFrame (or to a subset of it).
 * **Copy:** new DataFrame object (data is physically copied in memory).
   Modifying a copy leaves the original data untouched.

In general, **using `.loc[]` should return a view**, however that also depends on the evaluation order of some of the performed operations.

<br>

When you intend to make a copy, it is recommended to explicitly use the **`copy()`** method of DataFrame.

</div>

In [ ]:
# Explicitly create a copy of the DataFrame returned by .loc[]
df_male = df.loc[df.Sex == 'male', :].copy()

# Now we don't get a warning anymore:
df_male.Age = 888
df_male.head(3)

<br>
<br>
<br>

[Back to ToC](#toc)

# 2. Operations on columns <a id='4' />

## 2.1 Arithmetic and logical operations <a id='41' />

A great strength of pandas DataFrame is that it allows using **arithmetic operators directly on columns**.  
And as we have seen when doing conditional selection, this also works with **logical operators**, such as `>`, `<`, `==`, or `!=`.

> 🌈 **Note:** the usual shortcuts to increment/decrement a variable also apply here.
>
>   ```python
>   df["column"] += x  # Increment the column by `x`.
>   df["column"] -= x  # Decrement the column by `x`.
>   df["column"] /= x  # Divide the column by `x`.
>   df["column"] *= x  # Multiply the column by `x`.
>   ```

<br>

**Examples**:

* In the Titanic dataset, **increase the age of all passengers by 1 year**.

In [ ]:
# Print the DataFrame for reference.
df = pd.read_csv("data/titanic.csv").dropna()
df.Age = df.Age.astype(int)
df.head(3)

In [ ]:
# Increase everyone's age by 1 year.
df.Age = df.Age + 1
df.head(3)

<br>

**Examples using the Swiss 1880 census dataset:**
* The `Total` column gives the total number of registered inhabitants in a given town.
* The `Female` columns gives the number of women in a given town.

In [ ]:
# Load the Swiss 1880 census dataset.
df_census = pd.read_csv("data/swiss_census_1880.csv")
df_census.head(3)

<br>

* To compute the **fraction of women in each town**, we can simply write:

In [ ]:
df_census.Female / df_census.Total

<br>

* Better yet - we can very easily assign our result to a **new column**:

In [ ]:
df_census["Female_Fraction"] = df_census.Female / df_census.Total

# Display DataFrame subset of interest.
df_census.loc[:, ["town name", "Total", "Female", "Female_Fraction"]].head()

<br>

* We can test whether the dataset is consistent: the total number of women + men should
  equal the total population of a town.

> ✨ **Tip:** you can **invert a mask** (or any vector of boolean values) by prefixing it with **`~`**:
>
>    ```py
>    df.loc[~mask, :]
>    ```

In [ ]:
mask = df_census.Male + df_census.Female == df_census.Total

# Display the rows with errors, where the number of women + men does not match the total population.
df_census.loc[~mask, ["town name", "Total", "Male", "Female"]]

<br>

<div class="alert alert-block alert-success">

### Micro-Exercise 3

Reload the Titanic dataset with `df = pd.read_csv("data/titanic.csv")`, and perform the following task:
   
* Children under the age of 10 get a special discount of 50% on their fare.
  Apply this by dividing by 2 the `Fare` of eligible passenger in the `df` DataFrame.

<br>

🔮 **Additional task, if you have time:**
* Load the Swiss census 1880 dataset: `df_census = pd.read_csv("data/swiss_census_1880.csv")`.
* Create a mask to filter the dataset for towns that have a majority of Italian or Romansh speakers.
* In which cantons are these towns located, and how many are in each canton?

</div>

<br>
<br>

[Back to ToC](#toc)

## 2.2 Applying built-in summary functions <a id='42' />

The pandas `DataFrame` and `Series` objects (reminder: a single column of a `DataFrame` is a `Series`) have a number of **built-in methods to compute summary statistics on a per column (or per row) basis**.

* **`.count()`**: number of non-NA values.
* **`.value_counts()`**: count the number of occurrences of each value (works better with categorical data).
* **`.sum()`**, **`.mean()`**, **`.max()`**, **`.min()`**: sum/mean/max/min values.
* **`.std()`**, **`.var()`**: standard deviation and variance values.
* **`.round()`**: rounds values to the specified decimal.
* **`.all()`/`.any()`**: returns `True` if all elements/at least one element are `True` (truthy), `False` otherwise.
* **`.describe()`**: provide a summary of different statistics (mean, median, max, min, etc...).
* ... and more

**By default, these statistics are computed per column (`axis=0`).** To compute them per row, the `axis=1` argument must be passed.
* `axis=0`: apply the operation on columns (this is the default value).
* `axis=1`: apply the operation on rows.


<br>

**Examples:**
* Compute the mean of an individual column (pandas `Series`).

In [ ]:
df = pd.read_csv("data/titanic.csv")

# Average passenger age:
df["Age"].mean()

<br>

* Get the number of passengers in each class.

In [ ]:
df[["Pclass"]].value_counts()

<br>

* Retrieve the maximum value of the `Age` and `Fare` columns.

In [ ]:
df[["Age", "Fare"]].max()

<br>

Sometimes what we want is not the maximum/minimum value itself but **the index at which the maximum/minimum value is found** (e.g. to retrieve the entire row corresponding to that max/min value).  We can get this using:
* **`.idxmax()`** to retrieve the index of the row with the maximum value.
* **`.idxmin()`** to retrieve the index of the row with the minimum value.

In [ ]:
# Get the row of the passenger that paid the highest ticket price.
print("Index of row with max value of 'Fare':", df.Fare.idxmax())

df.loc[[df.Fare.idxmax()], ]

> ✨ **Tip:** to prevent the selection of a single row to be converted to a *Series*, we can pass the
> single element we select as a list with one element (in this case `[df.Fare.idxmax()]` instead
> of `df.Fare.idxmax()`).
> 
> ```python
> df.loc[[df.Fare.idxmax()], ]
> ```

<br>

---

<br>

A very useful method of DataFrame to get an **overview of a dataset** is **`df.describe()`**:

In [ ]:
df.describe()

* The `df.describe()` method gives information about all numerical columns in the dataset at once
  (note that by default, non-numerical columns are absent).
* It is very useful not only to get a first impression on the dataset, but also to catch eventual errors
  in the data: a negative number where there should be only positive values, missing values (NAs), ...


> *Note:* by default, `.describe()` only gives a summary of **numeric columns**.
> To include all columns in the summary table:
>
> ```py
> df.describe(include="all")
> ```


<br>

<br>

Anyway, back to the matter at hand.

**`.describe()`** gives access to some of the most commonly used summary statistics:
* (arithmetic) **mean**: ${\displaystyle \bar{x}={\frac {1}{n}}\sum _{i=1}^{n}x_{i}}$  or,
  for coders: `sum(x) / len(x)`
* **standard deviation** (a.k.a. std, stdev): this corresponds to the average of the absolute difference to the mean. It is the **square root of the variance**.
* **minimum** and **maximum**: smallest and biggest value among the data. Looking at them can help detect outliers.
* **quartiles**: they correspond to the value such that
    * 25% (first quartile, Q1), 
    * 50% (second quartile, Q2, median), or
    * 75% (second quartile, Q3)
    
  of the values are lower than them. They are less sensitive than the mean to outlier values.


<br>
<br>

[Back to ToC](#toc)

## 2.3 Applying custom functions <a id='43'/>

### Apply a custom function to each elements of a DataFrame or a Series (column of a DataFrame)

* **`.map(<function to apply>)`**: applies a given function to **each element** of a `DataFrame`/`Series`.


<br>

**Example:**

* Let's start by creating a custom function. **It is important that this function takes exactly 1 argument** (to be specific, no more than 1 positional argument, additional optional keyword arguments can be present).

In [ ]:
import math

# Declare a custom function the computes the log10 of a value.
def log10(x):
    """Return the log10 of the input value."""
    return 0 if x <= 0 else math.log10(x)

# Test run of our custom function.
for x in (0, 1, 10, 100, 1000):
    print(f"{x} -> {log10(x)}")

<br>

* We now **apply our `log10` function to (each element of) the `Fare` column** in the Titanic dataframe.

In [ ]:
df = pd.read_csv("data/titanic.csv")

df["Log10_fare"] = df.Fare.map(log10)
df.head()

<br>


* **`.map()` also works on entire DataFrames**: it will apply the custom function to each element
  of the DataFrame.

In [ ]:
df[["Age", "Fare", "Pclass"]].map(log10).head()

<br>

> 🌈 **Note:** if the custom function we want to pass is short and simple, we can write it as a
> **lambda expression** (i.e. anonymous function) rather than defining a function for it.
>
>   ```python
>   # The same log10 function as we used above.
>   df[["Age", "Fare", "Pclass"]].map(lambda x: 0 if x <= 0 else math.log10(x))
>
>   # A simple function that computes the square of a value.
>   df[["Age", "Fare", "Pclass"]].map(lambda x: x**2)
>   ```

<br>

<div class="alert alert-block alert-info">
    

### Additional material: applying custom functions to a column as a whole (function takes entire column as input)

* **`.apply()`**: applies a custom function to the columns of a DataFrame as a whole.
* The custom function must accept a single positional argument that expects a **sequence of values**.
* **Warning:** the `.apply()` method also exists for `Series`, but it applies the specified
  function to each element of the Series. In other words, it behaves just like `.map()` does on DataFrame.


**Example:**
    
```python
def sum_of_squares(values):
    """Computes the sum of squares of a sequence of values."""
    return sum([x**2 for x in values])


df_subset = df.dropna()                                     # Create a new DataFrame with no NA values.
df_subset[["Age", "Fare", "Pclass"]].apply(sum_of_squares)  # Apply custom function by column.

# Returns the following Series:
#  Age       7.743160e+05
#  Fare      2.843380e+06
#  Pclass    4.071000e+03
#  dtype: float64
```

* Functions can also be applied to **by row** by passing **`axis=1`** (the function is applied to the row as a whole).

```python
df_subset[["Age", "Fare", "Pclass"]].apply(sum_of_squares, axis=1) 
```

</div>

<br>

<div class="alert alert-block alert-warning">
    
### Important: vectorization vs. loops

When processing data in a `DataFrame`, you should **avoid using loops** as much as possible.  
Instead, use the pandas vectorization functions: `.map()`, `.apply()`.
    
Here is an example of a simple benchmark, where the **vectorized version is 100x faster** than using a loop.  
> 🌈 **Note:** `%timeit` is a Jupyter "magic function" that allows to benchmark a line of code.

<br>

```python
# Define a function that updates a DataFrame using a loop -> an example of what NOT TO DO.
def loop_method(df):
    abbreviations = {"S": "Southampton", "C": "Cherbourg", "Q": "Queenstown"}
    for index, value in enumerate(df["Embarked"]):
        df.loc[index, "Embarked"] = abbreviations.get(value, None)

# Define a function that does the same `loop_method`, but using vectorization.
def vectorized_method(df):
    df.Embarked = df.Embarked.map({"S": "Southampton", "C": "Cherbourg", "Q": "Queenstown"})

# Let's benchmark our two implementations.
df = pd.read_csv("data/titanic.csv")
%timeit loop_method(df)               # -> 60.3 ms ± 1.38 ms per loop
df = pd.read_csv("data/titanic.csv")
%timeit vectorized_method(df)         # -> 534 µs ± 23.2 µs per loop. More than 100x faster!
```

<div>

<br>

<div class="alert alert-block alert-success">

### Micro-Exercise 4

Here is a function that expands the abbreviated value for "Port of embarkation" (column name `Embarked`) in the Titanic dataset to the full name of the city ("C" for "Cherbourg", "Q" for "Queenstown", "S" for "Southampton").

```python
def expand_port_of_embarkation(input_value):
    """Converts the abbreviated port of embarkation to its full name."""
    abbreviations = {"C": "Cherbourg", "Q": "Queenstown", "S": "Southampton"}
    return abbreviations.get(input_value, None) if len(str(input_value)) == 1 else input_value
```

<br>

* Using the above function, your task is to **add a new `Embarked_city` column** to the DataFrame that contains the full name
  of the port of embarkation of each passenger.  
* If needed, you can reload the Titanic dataset with: `df = pd.read_csv("data/titanic.csv")`

</div>

<br>
<br>

[Back to ToC](#toc)

# 3. Grouping data by factor <a id='a3' />
---------------------

When analyzing a dataset where some variables (columns) are factors (categorical values), it is often useful to group the samples (rows) by these factors.  
This can be done using the method:

* **`.groupby()`**: group data by one or more categorical columns.


<br>

**Example:** grouping by a **single factor**.

* We earlier computed the proportions of women and men that survived in the Titanic dataset.
  Using **`.groupby()`** can make this a lot easier.

In [ ]:
df = pd.read_csv("data/titanic.csv")
df.groupby("Sex")

The operation creates a groupby object on which we can call summary functions (*eg,* `.sum()`, `.mean()`) or a custom function

> *Note:* since a mean value can only be computed for numeric values, the argument `numeric_only` must be
> set to `True` so that any non-numeric column gets skipped.

In [ ]:
df.groupby("Sex").mean(numeric_only=True)

We can also do it for a specific column:

In [ ]:
df.groupby("Sex").Age.median()

<br>

**Example:** grouping by **multiple factors**.

* Compute mean values by gender and passenger class.

In [ ]:
df.groupby(["Sex", "Pclass"]).mean(numeric_only=True)

<div class="alert alert-block alert-success">

### Micro-Exercise 5

* Compute survival rates by gender, and by passenger class.

<div>

<br>
<br>
<br>

[Back to ToC](#toc)


# 4. Writing DataFrames to disk <a id='5'/>
----------------------------------------------------------

Just like when reading a file, **writing a DataFrame to disk** has several functions depending on the format in which the data should be stored.
* **`to_csv()`**: write a DataFrame as comma-separated file, or any other separator-delimited format such as
  tab-delimited. 
* **`to_excel()`**: write a DataFrame in Excel format.
* **`to_html()`**: write a DataFrame in HTML format.
* See [here for more writer functions...](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

These functions take similar arguments as the reader functions. E.g. when using the `to_csv()` writer functions, some useful arguments are:

* **`sep`**: the type of delimiter to use. By default, `sep=","`.
  To write a tab-delimited file e.g., one would set `sep="\t"`.
* **`header=None`**: to not include the header in the exported file.

<br>

**Example:** write the `df` DataFrame to a file named `my_data.csv` in the current working directory.

In [ ]:
df.to_csv("my_data.csv")

<br>
<br>
<br>

<div class="alert alert-block alert-success">

# Exercises
-----------------

Exercises are located in the dedicated notebook `exercises_course1.ipynb`.

* **Exercise 1.1**
* If you have time: **additional exercises 1.2 and 1.3**


</div>


<br>
<br>
<br>
<br>

[Back to ToC](#toc)

# Supplementary material - additional topics, not covered in class <a id='a' />
---------------------------

Pandas is a very large library, and we can only cover part of it during the class.
Here is a **collection of additional topics that are useful to know**.  
This section will not be covered in the course, but you can read it on your own if you are interested.

Topics presented in the supplementary material include (but are not limited to):

* **Sorting** operations on dataframes.
* **Dropping rows** with missing values.
* **Grouping data** by factor.
* **Creating** DataFrames and Series **from scratch**.
* **Concatenate**, **merge** and **join** DataFrames.


<br>

## Annex 1 - Sorting operations on dataframes  <a id='a1' />

DataFrames can be sorted using:

* **`sort_values()`**: to sort rows based one or more column(s).
  * The name of the column to sort on must be passed to the function.
  * To sort on multiple columns, a list of column names must be passed.
* **`sort_index()`**: to sort rows based on the dataframe index.

Both functions take the optional arguments:
* `ascending=True`: sort in ascending (`True`, the default) or descending (`False`) order.
* `inplace=False`: if `True` the original DataFrame is sorted, if `False` a sorted copy
  of the DataFrame is returned (this is the default).

<br>

**Examples:**

In [ ]:
# Load dataset with Unique Molecular Identifier (UMI) counts per gene.
df = pd.read_table('data/pbmc_data.countMatrix.50.txt.zip', sep=" ", index_col=0)
df.head()

<br>

* Sort the `df` DataFrame in descending order of the first column.  
> 🌈 **Note:** to avoid having to pass the explicit column name, we use `df.columns[0]`.

In [ ]:
df = df.sort_values(df.columns[0], ascending=False)
df.head()

<br>

* Sort the `df` DataFrame in descending order of the first 3 columns.

In [ ]:
df = df.sort_values(list(df.columns[0:3]), ascending=False)
df.head(10)

<br>

* Sort the DataFrame by index values using **`sort_index()`**:

In [ ]:
df.sort_index(ascending=True).head()

<br>
<br>

[Back to ToC](#toc)

## Annex 2 - Dropping rows with missing values (`NaN`) <a id='a2' />

Datasets frequently contain rows with missing data, indicated as `NaN` or `NA` (stands for "not a number" - but it is used even if the column type is not numeric).

In the titanic dataset for instance, `NaN` values are found in the columns `Age` and `Embarked`:

In [ ]:
df = pd.read_csv("data/titanic.csv")
df.head(100).tail()

To remove rows with `NaN` values, the **`dropna()`** method of a DataFrame can be used:
* By default, `dropna()` drops rows containing `NaN` values. Columns can be dropped by passing `axis=1`
  to the function.
* By default, the method returns a **copy of the DataFrame**. Use `dropna(inplace=True)` to modify
  the original DataFrame.

In [ ]:
df.dropna().head(80).tail()
print("Number of rows in input DataFrame:", df.shape[0])
print("Number of rows after removing NaN values:", df.dropna().shape[0])

<br>
<br>

[Back to ToC](#toc)

## Annex 4 - Creating DataFrames and Series from scratch <a id='a4' />

### Creating DataFrames

To create a new pandas DataFrame, we pass a `dict` (dictionary) to **`pd.DataFrame()`**, where:

* The dictionary's **keys are column names**.
* The dictionary's **values are the values for the given column**. The values can be either:
    * A sequence - e.g. a `list` or a `tuple`.
    * A unique value - in which case all values in the column will be identical.
  
By default, a numeric index is auto-generated with values starting at `0` (corresponding to row positions).  
A custom index can be set by adding the `index` argument when instantiating the DataFrame, or the default index can be replaced by a custom index after the DataFrame is created.

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "Color": ("red", "green", "pink", "green"),
        "Speed": [8, 7, 5, 5],
        "Jump": [7, 8, 7, 5]
    },
    index=["Mario", "Luigi", "Peach", "Yoshi"]
)

# Note: evaluating a DataFrame object at the end of a jupyter cell will render it nicely in the notebook.
df

### Creating a pandas Series <a id='1.2'></a>

To create a new pandas **Series**, we pass a sequence (e.g. list, tuple, dict, generator) to **`pd.Series()`**:
* The optional `name` argument allows to associate a "name" to the Series.
* As with `pd.DataFrame()`, an optional `index` argument can be passed (by default the index
  is set to numerical values starting from `0`).
* Alternatively, a `dict` can be passed as input to `pd.Series()`, in which case the keys of the dictionary
  are used as index values, and the values as values for the Series.
  
The basic characteristics of a Series (here named `s`) are:
* Its **length**: retrieved with **`s.size`** or `len(s)`
* Its **name**: retrieved with **`s.name`**.

<br>

**Example:**

In [ ]:
new_player = pd.Series(
    ["Toad", "white", "10"],
    index=["Color", "Speed", "Jump"],
    name="Toad",
)

new_player

<br>

### Adding a Series as a row to a DataFrame with `pd.concat()`

**`pd.concat()`** is a method generally used to concatenate DataFrames (either along rows or columns). But it can also be used to add a Series as a new row to an existing DataFrame.

Note that:
* We are actually converting the Series to DataFrame with `.to_frame()`, and transpose it with `.T`.
* The function does **not modify the existing DataFrame/Series** given in input, but **returns a new one**.
* For the concatenation to work, the **index of the Series must correspond to the column names of the 
  DataFrame**.

In [ ]:
pd.concat((df, new_player.to_frame().T))

<br>
<br>

[Back to ToC](#toc)

## Annex 5 - Concatenate, merge and join DataFrames  <a id='a5' />

### DataFrame concatenation

The **`.concat()`** method allows to **concatenate two or more DataFrames**.  
The `axis` argument allows to specify whether the concatenation should be along rows or columns.

* **`.concat([df1, df2, ...], axis=0)`**: concatenate rows of the DataFrames. This is the default.
* **`.concat([df1, df2, ...], axis=1)`**: concatenate columns of the DataFrames.

<br>

**Examples:**

* Let's start by creating a few small DataFrames.

In [ ]:
df1 = pd.DataFrame(
    {
        "Color": ["red", "green", "pink", "green"],
        "Speed": [8, 7, 5, 5],
        "Jump": [7, 8, 7, 5]
    },
    index=["Mario", "Luigi", "Peach", "Yoshi"]
)
df2 = pd.DataFrame(
    {
        "Color": ("white", "yellow"),
        "Speed": [10, 3],
        "Jump": [7, 3]
    },
    index=["Toad", "Bowser"]
)
df3 = pd.DataFrame(
    {
        "Strength": [5, 10, 6],
        "Groove": [7, 1, 10],
    },
    index=["Mario", "Bowser", "Yoshi"]
)

print(df1)
print(df2)
print(df3)

<br>

* We can now concatenate `df1` and `df2` by rows.

In [ ]:
pd.concat([df1, df2])

#### `"outer"` vs. `"inner"` joining

* By default, `.pd.concat()` uses the value **`join="outer"`** to concatenate DataFrames.
  This means that all rows/columns of both DataFrames are kept, and `NaN` values are inserted
  in missing fields. In other words, it takes the **union** of both DataFrames.

In [ ]:
pd.concat([df1, df3], axis=1, join="outer")

* Using **`join="inner"`** only keeps rows with an index that is present in both DataFrames.
  In other words, it takes the **intersection** of both DataFrames.

In [ ]:
pd.concat([df1, df3], axis=1, join="inner")

<br>

### Merge and join

The **[`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)** and **[`join()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html)** methods allow to combine DataFrames, linking their rows based on a common column (also referred to as a **key**).

To illustrate these 2 methods, let's create 2 DataFrames that we can merge.
> 🌈 **Note:** this also illustrates how **a dataframe can be constructed from a dictionary** data structure.
> * The dictionary keys are treated as column names, and the list of values associated with a key is
>   treated as list of elements in the corresponding column. Note that all columns should have the same
>   number of elements (or a single element, in which case all rows of the column are assigned this same
>   element).  
> * If no index is specified, pandas uses its default indexing, i.e. row positions.

In [ ]:
df1 = pd.DataFrame(
    {
        "key": ["b", "b", "a", "c", "a", "a", "b"], 
        "data1": range(7)
    }
)
df1

In [ ]:
df2 = pd.DataFrame({
    "key": ["a", "b", "d"], 
    "data2": range(3)
})
df2

<br>

**Let's now merge the two data frames**, with the default application of the **`.merge()`** method:

In [ ]:
pd.merge(df1, df2)

How has python interpreted our call to `.merge()`?

1. It has assumed that we want to merge on the basis of the common `key` column.
2. It has identified the values of `key` which occur in both dataframes.
3. It has generated a dataframe with all combinations of rows from dataframes 1 and 2 that are 
   associated with a particular `key` value.

We can be more precise by specifying how to merge the dataframes, using the **`on`** option:

In [ ]:
pd.merge(df1, df2, on="key")

<br>

**By default, `.merge` performs an "inner" operation**, taking the intersection of the key sets. However, we can specify the way we want to merge by passing `"outer"`, `"left"`, `"right"` to the **`how`** argument. This determines which set of keys to consider (the union of the two sets, all of those that occur in df1, all of those that occur in df2). Missing values show up as `NaN`.

In [ ]:
pd.merge(df1, df2, on="key", how="outer")

<br>

**Merging can also be done based on the index values**. Let's illustrate this using another dataset:

* First let's create 2 DataFrames to merge: `df_A` and `df_B`.

In [ ]:
df = pd.read_table('data/pbmc_data.countMatrix.50.txt.zip', sep=" ", index_col=0)

# Split the columns based on whether or not they start with "AAAT".
mask = df.columns.str.startswith("AAAT")
AAAT_cols = df.columns[ mask ]
nonAAAT_cols = df.columns[ ~mask ]

df_A = df[AAAT_cols]
df_B = df[nonAAAT_cols]

df_A.head()

In [ ]:
df_B.head()

<br>

* Merge the two DataFrames based on the index:

In [ ]:
merged_df = pd.merge(df_A, df_B, left_index=True, right_index=True)
merged_df.head()

<br>
<br>

[Back to ToC](#toc)

## Annex 6 - Mixed selection by names and positions for `.loc[]` and `.iloc[]` <a id='a6' />
    
A frequent situation is that we want to select rows based on a certain condition, e.g. `df["Age"] <= 35`, 
and columns based on position , e.g. `1:3` to select the second and third columns. The problem is then the following:
* **`.loc[]`** does not support column selection by position: it requires index values, i.e. column
  and row names.
* **`.iloc[]`** does not support boolean results (`True`/`False`) for row selection: it requires to get
  a position.
  
Possible solutions to mixed indexing are:

* Use the **`.columns` attribute** to get the names of columns, which can then be used with `.loc[]`.
  ```python
  df.loc[ df["Age"] <= 35, df.columns[1:3] ]
  ```
* If the **index values correspond to row positions (0, 1, 2, ...)**, the `.index` attribute can be 
  used to get row positions and use them with `.iloc[]`:
  ```python
  df.iloc[ df[df["Age"] <= 35].index, 1:3 ]
  ```
* With `.iloc[]`, use the **`.query()` method** to select rows:
  ```python
  df.iloc[:, 0:6].query("Age <= 35")
  ```

<br>

**Examples:**
    
</div>

In [ ]:
# Select passengers younger than 35 and the 5 first columns:
df = pd.read_csv("data/titanic.csv")
df.loc[df["Age"] < 35, df.columns[0:5]].head()

In [ ]:
# Same as above, but using the "iloc[]" indexer:
df.iloc[:, 0:6].query("Age < 35").head()

<br>
<br>

[Back to ToC](#toc)

## Annex 7: more ways to add rows to a DataFrame <a id='a7' />

Several options exist to add or edit rows of a DataFrame:

* As we have already seen, the easiest way is to **assign a sequence of values to a new row**
  (or an existing one to overwrite it).  
  **Important:** the sequence must have the same length as there are columns in the DataFrame.


In [ ]:
df = pd.read_csv("data/titanic.csv")

# Add a new row to the DataFrame.
df.loc[len(df),:] = ["Bob", "male", 27, 3, 1, 0, 10, "S"]
df.tail()

<br>

A row can also be added by **assigning a `Series` to a new row** (or an exiting row, to overwrite it).  
When using this method:
* The names used in the Series' index must match with the column names.
* Missing fields are allowed.
* The order of fields has no importance, since they are named.


In [ ]:
new_passenger = pd.Series({"Name": "Alice", "Sex": "female", "Embarked": "C", "Fare": 95})
df.loc[len(df),:] = new_passenger
df.tail()

<br>

Another option is to **concatenate a DataFrame** with another `DataFrame` or `Series` using  **`pd.concat()`**, which takes a list of `DataFrame` or `Series`.


In [ ]:
pd.DataFrame(
            {
                "Name": "Chuck Norris",
                "Sex": "fluid",
                "Age": pd.NA,
                "Pclass": 2,
                "Survived": 2,
                "Family": pd.NA,
                "Fare": 0,
                "Embarked": "S"
            },
        index=[df.shape[0]])

In [ ]:
df = pd.concat(
    [
        df,
        pd.DataFrame(
            {
                "Name": "Chuck Norris",
                "Sex": "fluid",
                "Pclass": 2,
                "Survived": 2,
                "Fare": 0,
                "Embarked": "S"
            },
        index=[df.shape[0]])
    ]
)
df.tail()

<br>

**Adding a row at specific location** is possible, if a little hacky as you first have to add the row at the
end of the DataFrame, and then re-order the rows.


In [ ]:
df.loc[890.5,:] = ["Bob Jr.", "male", 2, 3, 1, 1, 10, "S"]
df.sort_index(inplace=True)
df.reset_index(inplace=True, drop=True)  # With drop=True, the index is reset to the default index values.
df.tail()

# Alternatively: df = df.sort_index().reset_index(drop=True)

<br>
<br>

[Back to ToC](#toc)

## Annex 8 - Wide and long format <a id='a8' />

Most dataset are usually stored in so-called **wide format**, because it is more efficient and avoids duplication of information in the table. However, it can sometimes be interesting to go from wide to long or long to wide, because some operations are easier on one format or the other.

**Wide format:**

| Id     | mol1   | mol2    |
| ------ |:------:| -------:|
| a      | 1.0    | 10.0    |
| b      | 2.0    |   20.0  |
| c      | 3.0    |    30.0 |

**Long format:**

| Id     | Value  | Attr    |
| ------ |:------:| -------:|
| a      | 1.0    | mol1    |
| a      | 10.0   | mol2    |
| b      | 2.0    | mol1    |
| b      | 20.0   | mol2    |
| c      | 3.0    | mol1    |
| c      | 30.0   | mol2    |



In [ ]:
df = pd.read_table("data/pbmc_data.countMatrix.50.txt.zip", index_col=0,sep=' ')

## data in wide format.
df.head()

<br>

The conversion to the **long** format is done using the **`.melt()`** method of DataFrame.

In [ ]:
df["gene"] = df.index
df_long = pd.melt(df, id_vars=["gene"])  # Convert to long format, with gene as identifiers.
df_long.head()

<br>

Having the data in long format now allows us to do this, which was a bit difficult otherwise:  
> 🌈 **Note:** running the cell below can take a few seconds (notice the "[ \* ]" to the left of the Jupyter Notebook cell, indicating that the cell is computing).

In [ ]:
import numpy as np
import seaborn as sns

df_long["logVal"] = np.log10(10**0 + df_long["value"])

g = sns.catplot(
    x="logVal", y="variable",
    orient="horizontal" , data=df_long, 
    kind="bar", aspect=2, height=7
)